In [ ]:
import pulp
import pandas as pd

In [ ]:
prob = pulp.LpProblem("Test_plan", pulp.LpMinimize)

In [ ]:
df = pd.read_csv("data.csv")

In [ ]:
labels = list(df['Food'])

In [ ]:
costs = dict(zip(labels,df['Cost']))

In [ ]:
daily = dict(zip(labels,df['Daily']/1000))
weekly = dict(zip(labels,df['Weekly']/1000))

In [ ]:
foods_mon = pulp.LpVariable.dicts("Food_Mon",labels,0)
foods_tue = pulp.LpVariable.dicts("Food_Tue",labels,0)
foods_wed = pulp.LpVariable.dicts("Food_Wed",labels,0)
foods_thu = pulp.LpVariable.dicts("Food_Thu",labels,0)
foods_fri = pulp.LpVariable.dicts("Food_Fri",labels,0)
foods_sat = pulp.LpVariable.dicts("Food_Sat",labels,0)
foods_sun = pulp.LpVariable.dicts("Food_Sun",labels,0)
food_list = [foods_mon,foods_tue,foods_wed,foods_thu,foods_fri,foods_sat,foods_sun]

In [ ]:
prob += pulp.lpSum([costs[food]*foods_mon[food]+costs[food]*foods_tue[food]+costs[food]*foods_wed[food] for food in labels])

In [ ]:
for food in labels:
    for foods in food_list:
        prob += foods[food] <= daily[food]
    prob += sum([foods[food] for foods in food_list]) <= weekly[food]

In [ ]:
energy = dict(zip(labels,df['Energy']*10))
fat = dict(zip(labels,df['Fat']*10))
saturates = dict(zip(labels,df['Saturates']*10))
carbohydrates = dict(zip(labels,df['Carbohydrates']*10))
sugars = dict(zip(labels,df['Sugars']*10))
protein = dict(zip(labels,df['Protein']*10))

In [ ]:
for foods in food_list:
    prob += pulp.lpSum([energy[food] * foods[food] for food in labels]) >= 2000*0.9
    prob += pulp.lpSum([energy[food] * foods[food] for food in labels]) <= 2000*1.1
    prob += pulp.lpSum([fat[food] * foods[food] for food in labels]) >= 70*0.9
    prob += pulp.lpSum([fat[food] * foods[food] for food in labels]) <= 70*1.1
    prob += pulp.lpSum([saturates[food] * foods[food] for food in labels]) >= 20*0.9
    prob += pulp.lpSum([saturates[food] * foods[food] for food in labels]) <= 20*1.1
    prob += pulp.lpSum([carbohydrates[food] * foods[food] for food in labels]) >= 260*0.9
    prob += pulp.lpSum([carbohydrates[food] * foods[food] for food in labels]) <= 260*1.1
    prob += pulp.lpSum([sugars[food] * foods[food] for food in labels]) >= 90*0.9
    prob += pulp.lpSum([sugars[food] * foods[food] for food in labels]) <= 90*1.1
    prob += pulp.lpSum([protein[food] * foods[food] for food in labels]) >= 50*0.9
    prob += pulp.lpSum([protein[food] * foods[food] for food in labels]) <= 50*1.1

In [ ]:
prob.solve();

In [ ]:
print(pulp.LpStatus[prob.status])

In [ ]:
print('Cost =', pulp.value(prob.objective), '€')

In [ ]:
for day in ["_Mon_","_Tue_","_Wed_","_Thu_","_Fri_","_Sat_","_Sun_"]:
    print()
    for v in prob.variables():
        if v.varValue and day in v.name:
            print(v.name, '=', '{0:.0f}'.format(v.varValue*1000), 'g')